# Create convolutional neural network model

## Imports

In [ ]:
#import numpy as np
#import h5py
from tflearn import DNN
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
from tflearn.data_utils import image_preloader

## Get dataset

In [ ]:
# Load path/class_id image file:
data_folder = 'images/final_dataset'

# Build the preloader array, resize images to 480 x 640
X, Y = image_preloader(data_folder, image_shape=(320, 240), mode='folder', categorical_labels=True, normalize=True, files_extension=['.jpg'])

# Show shape and features
print("image shape: ", X[0].shape, "Features: ", len(Y[0]))

from matplotlib.pyplot import imshow
%matplotlib inline

imshow(X[0])

### Image augmentation
We use image augmentation to create a better training effect for real world situations. We randomly blur images and rotate them.

In [ ]:
# Real-time data augmentation (This is only used while training the DNN)
img_aug = ImageAugmentation()
img_aug.add_random_blur(0.25)
img_aug.add_random_rotation(max_angle=10.0)

### Image preprocessing

In [ ]:
# Real-time data preprocessing
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

### Convolutional neural network model

In [ ]:
# Convolutional network building
network = input_data(shape=[None, 240, 320, 3],
                     data_augmentation=img_aug,
                    data_preprocessing=img_prep)
network = conv_2d(network, 16, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)
network = fully_connected(network, 64, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 7, activation='softmax')
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

model = DNN(network, tensorboard_verbose=0)

### Train the model and save it

In [ ]:
model.load('models/final_model_aug_prep')

# Start the trainig process
model.fit(X, Y, n_epoch=10, shuffle=True, validation_set=0.1, show_metric=True, batch_size=16, validation_batch_size=16, run_id='final_model_aug_prep')

# Evaluate performance on test set
#model.evaluate(X_test, Y_test)

# Save the model
model.save('models/final_model_aug_prep')

# Close the database connection
#h5f.close()